# Plotting Basics - IRIS-HEP Analysis Training 

### Histograms mean different things in different contexts
- counts, bin edges - useful for a bar plot - `np.histogram` / `plt.bar`
- counts, bin edges, pre computed errors - `TGraphErrors`/`plt.errorbar`
- weighted values, weights squared, bin_edges - proper error calculation `TH1`/`Coffea.hist`/`hist`

## UHI - [Unified Histogram Interface](https://uhi.readthedocs.io/en/latest/plotting.html#using-the-protocol)
- (Plottable) Histogram protocol - designed to make libraries interoperable, easy to navigate
  - Conformed to by `hits`, `mplhep`, `uproot4`, `histoprint`
- Each UHI histogram has the following methods
  - `h.values()`: The value (as given by the kind)
  - `h.variances()`: The variance in the value (None if an unweighed histogram was filled with weights)
  - `h.counts()`: How many fills the bin received or the effective number of fills if the histogram is weighted
  - `h.axes`: A Sequence of axes
  - and a few other properties

## [hist](https://github.com/scikit-hep/hist)
- python go to one-stop for histogramming
- extends [boost-histogram](https://github.com/scikit-hep/boost-histogram.html) (pythonic wrapper for C++ library - *FAST*)
  - makes it user friendly
- shortcuts for convenience - plotting/fitting

## [mplhep](https://github.com/scikit-hep/mplhep)
- build on top of `matplotlib`
- extends functionality to easily plot histograms from various inputs
- holds style sheets for easy experiment specific style application

# Outline

 - Short matplotlib info
 - Histogramming in matplotlib
 - `mplhep` - basic example
 - `hist` - basic examples and indexing with UHI
 - Analysis style example
 
 
 

# Two ways of matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Stateful

In [ ]:
plt.plot(np.arange(0, 10, 1), np.linspace(0, 1, 10))
# plt.title("Test")
# plt.legend()

### Object-oriented

In [ ]:
fig, ax = plt.subplots()

In [ ]:
ax.plot(np.linspace(0, 1, 10), np.linspace(0, 10, 10))

In [ ]:
fig

## Switching back and forth

In [ ]:
fig, ax = plt.subplots()
ax.stairs([1,2,3,4,3,2,1])
plt.title("TEST")

In [ ]:
plt.stairs([1,2,3,4,3,2,1])
ax = plt.gca()
ax.set_title("TEST")

# Histogramming in matplotlib

In [ ]:
fig, ax = plt.subplots()
ax.stairs([1,2,3,4,2,1,0])

In [ ]:
fig, ax = plt.subplots()
ax.stairs([1,2,3,4,2,1,0], baseline=0, fill=True)

In [ ]:
a, b = [1,2,3,4,2,1,0], [1,2,3,2,2,3,1]

fig, ax = plt.subplots()
ax.stairs(a, label="A")
ax.stairs(b, label='B', ls='--')
plt.legend()

In [ ]:
fig, ax = plt.subplots()
ax.stairs(np.sum([a,b], axis=0), baseline=b, fill=True, label="A")
ax.stairs(b, fill=True, label='B')
plt.legend()

## Other histogramming methods

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
axs = axs.flatten()

# in-situ np.histogram()
axs[0].hist(np.random.normal(5, 1, 10000));

# bar plots
# axs[1].bar([1,2,3], [2,3,4])  # (x-position), bin-value);

# step - skyline
# axs[2].step(np.arange(0, 5, 1), [2,3,4,2,1], where='post');

# filled
# axs[3].fill_between(np.arange(0, 5, 1), [2,3,4,2, 1], step='post');

# Better histogramming - mplhep

In [ ]:
import mplhep as hep

In [ ]:
yields, bins = np.histogram(np.random.normal(5, 1, 5000), bins=10)

In [ ]:
hep.histplot(yields)

### Primary goal is to stay unobtrusive, if it works in `matplotlib`, it should work in `mplhep`

In [ ]:
import mplhep as hep
f, axs = plt.subplots(1,2, figsize=(12, 4))

hep.histplot(yields, ax=axs[0])
hep.histplot(yields, bins, yerr=True, ax=axs[1]);

### Kwargs are passed though to `matplotlib`

In [ ]:
f, axs = plt.subplots(1,2, figsize=(12, 4))

hep.histplot(yields, ax=axs[0], histtype='fill', hatch='///', edgecolor='red', facecolor='none')
hep.histplot(yields, ax=axs[1], histtype='errorbar', yerr=True, c='black', capsize=4)

### Stacking, norming is available

In [ ]:
h = yields
f, axs = plt.subplots(1,3, figsize=(18, 4))

data = np.random.poisson(h*3)
hep.histplot([h, h*2], bins=bins, ax=axs[0], yerr=True, label=["MC1", "MC2"])
hep.histplot(data, bins=bins, ax=axs[1], yerr=True, label="Data")

hep.histplot([h, h*2], bins=bins, ax=axs[2], stack=True, label=["MC1", "MC2"], density=True)
hep.histplot(data, bins=bins, ax=axs[2], yerr=True, histtype='errorbar', label="Data", density=True, color='k')
for ax in axs:
    ax.legend()
axs[0].set_title("Some MCs")
axs[1].set_title("Draw Poisson Data")
axs[2].set_title("Data/MC Shape comparison"); 

### Convenient sorting options

In [ ]:
f, axs = plt.subplots(1,2, figsize=(12, 4))
hep.histplot([h*2, h*3, h], bins=bins, ax=axs[0], stack=True, histtype='fill', label=["A", "B", "C"], sort='yield');
hep.histplot([h*2, h*3, h], bins=bins, ax=axs[1], stack=True, histtype='fill', label=["A", "B", "C"], sort='label_r');
for ax in axs:
    ax.legend()
axs[0].set_title("Sort by yield")
axs[1].set_title("Sort by label (add _r to reverse)");

In [ ]:
## Uproot TH1 
import uproot
from skhep_testdata import data_path
fname = data_path("uproot-hepdata-example.root")
f = uproot.open(fname)
print(f.keys())
print(f['hpx'])
hep.histplot(f['hpx']);

In [ ]:
# PyROOT TH1
import ROOT
h = ROOT.TH1F("h1", "h1", 50, -2.5, 2.5)
h.FillRandom("gaus", 10000)

hep.histplot(h);

# Better histogramming with hist

In [ ]:
import hist

In [ ]:
# histogram creation
h = hist.Hist(
    hist.axis.Regular(10, 0, 10, name="x", label="x-axis"),
#     hist.axis.Variable([0, 1, 2, 5, 10], name="y", label="y-axis"),
    hist.storage.Int64()
)
h

In [ ]:
# basic filling
h.fill([1, 4, 6], 
#       [3, 5, 2]
      )
h

In [ ]:
# Filling by names is possible for better bookkeeping:
h.fill(x=[1, 5, 5, 7], 
#        y=[3, 5, 2]
      )
h

In [ ]:
# information access
h.values()

In [ ]:
h.axes[0]

In [ ]:
h.axes[0].edges

In [ ]:
# Print it (to CLI)
h.show(columns=50)

## Quick hist creation

In [ ]:
# histogram creation
h = hist.new.Regular(10, 0, 10, name="x", label="x-axis") \
    .Variable(range(10), name="y", label="y-axis") \
    .Int64().fill(*np.random.multivariate_normal([4, 6], [[2, 0], [0, 1]], 10000).T)
    
h

In [ ]:
# even quicker
h = hist.new.Reg(10, 0, 10).Var(range(10)).Int64() \
#     .fill(*np.random.multivariate_normal([4, 6], [[2, 0], [0, 1]], 10000).T)
h

## Axis types 

In [ ]:
axis0 = hist.axis.Regular(10, -5, 5, overflow=False, underflow=False, name="A")
# axis1 = hist.axis.Boolean(name="B")
axis2 = hist.axis.Variable(range(10), name="C")
# axis3 = hist.axis.Integer(-5, 5, overflow=False, underflow=False, name="D")
axis4 = hist.axis.IntCategory(range(10), name="E")
axis5 = hist.axis.StrCategory(["T", "F"], name="F")

In [ ]:
# Growth!
h = hist.new.Reg(10, 0, 10).StrCat([], growth=True).Weight()
h.fill(np.random.normal(5, 2, 1000), "A")
h.fill(np.random.normal(7, 2, 1000), "B")

## Storage types

A number of possible storage type exist: `Double`, `Unlimited`, `Int64`, `AutomicInt64`, `Weight`, `Mean`, and `WeightedMean`.

In practice you will always use `Weight()`

In [ ]:
hist.new.Reg(10, 0, 10).Weight().fill([1,2,3,5], weight=[1,1,1,0.5])

In [ ]:
hist.new.Reg(10, 0, 10).Weight().fill([1,2,3,5])

## Hist manipulation and UHI

https://hist.readthedocs.io/en/latest/user-guide

Yes, we've had one documentation, but what about another one...

https://uhi.readthedocs.io/en/latest/

In [ ]:
# example histogram
h = hist.new.Reg(10, 0, 10, name="x") \
    .Var(range(10), name="y") \
    .Var(range(10), name="z") \
    .Weight().fill(*np.random.multivariate_normal([4, 6, 4], np.eye(3), 100000).T)
    
h

In [ ]:
# Project on an axis
h.project("x")

In [ ]:
# Slicing (applying cuts)
h[5:, :, sum]

In [ ]:
# Indexing by bin vs value
h[5:, 5j:, sum]

In [ ]:
# Dictionary access
h[{"y": 5, "z": sum}]

In [ ]:
# More robust slicing and  
s = hist.tag.Slicer()
s

In [ ]:
h[{"z": sum, "y": s[:hist.loc(5):hist.sum]}]

## Mind the flow bins!

In [ ]:
h[sum, sum, :].values()

In [ ]:
h[sum, sum, :].values(flow=True)

In [ ]:
h[sum, 0:len:sum, :].values(flow=True)

In [ ]:
h[sum, sum, :].values(flow=True) - h[sum, 0:len:sum, :].values(flow=True)

In [ ]:
# Doesn't work in dict-access
# h[{0: 0:len:sum}] 

In [ ]:
# Meanwhile slicer allows this syntax in dict-access
h[{0: s[0:len:sum]}]

In [ ]:
# If you know you won't need them, you can skip flow bins
hist.new.Reg(10, 0, 10, flow=False).Weight()

 ## Hist plots with mplhep

In [ ]:
h = hist.new.Reg(10, 0, 10).Weight().fill(np.random.normal(5, 1, 1000))

In [ ]:
# Plot it
h.plot(color='red', density=True);

In [ ]:
#equivalent to  
hep.histplot(h, color='red', density=True)

In [ ]:
# Access and modify artists
art = h.plot(color='red', density=True);
plt.setp(art[0].stairs, edgecolor='blue', fill=True, facecolor='lightgreen', hatch='///');

## N-D Histograms are cool

In [ ]:
# Create a new hist
h2d = hist.new.Reg(10, 0, 10, name='x').StrCat(["A", "B"], growth=True, name='dataset').Weight()
h2d

In [ ]:
# Fill it
h2d.fill(np.random.normal(3, 1, 1000), "A")
h2d.fill(np.random.normal(5, 1, 3000), "B")
h2d.fill(np.random.normal(7, 1, 2000), "C")
h2d

In [ ]:
h2d[:6, ["A", "B"]].plot(stack=True, histtype='step', sort='y_r');
plt.legend()

In [ ]:
hep.hist2dplot(h2d, labels=True);

# Analysis-like example

In [ ]:
hn = (hist.new.Reg(100, 0, 100, name='x', label='Observable')
      .Var([0, 0.2, 0.5, 0.9, 1], name="tag", label="Some MVA")
      .StrCat(["A"], growth=True, name='dataset')
      .IntCat([0, 1, 2, 3], name='region')
      .StrCat(["A"], growth=True, name='syst', label='Systematic')
      .Weight()
     )

In [ ]:
# Small random letter helper
def rnd_letters(a="A", z="Z", N=10):
    A, Z = np.array([a, z]).view("int32") 
    return list(np.random.randint(low=A,high=Z,size=N,dtype="int32").view(f"U{N}")[0])
rnd_letters("C", "F")

In [ ]:
# And fill it
N = 400000
for sample in set(rnd_letters("A", "G", 500)):
    hn.fill(x = np.random.normal(np.random.randint(20, 80, 1), 10, N),
            tag = np.random.uniform(0, 1, N),
            dataset = sample,
            region=np.random.randint(0, 4, N),
            syst = rnd_letters("P", "Z", N=N)
            )
hn

In [ ]:
# Simple slices
hn[:, 0.5j:len:sum, :, 0, "X"]

In [ ]:
# Slice by name
s = hist.tag.Slicer()
hn[{'tag': s[0.5j:len:sum], 'region': 0, 'syst': "X"}].plot();
plt.legend();

### Scale "sample" by "cross-section"

In [ ]:
hn[{'dataset': "A"}] = hn[{'dataset': "A"}].view() * 2.5

hn[{'tag': s[0.5j:len:sum], 'region': 0, 'syst': "X"}].plot();
plt.legend();

### Group datasets (to be replaced by native hist function)

In [ ]:
def groupby(h, groupmap, axis='dataset'):
    new = hist.Hist(*[ax for ax in h.axes if ax.name != axis], 
                hist.axis.StrCategory(groupmap.keys(), name=axis, growth=True), 
                hist.storage.Weight()
          )

    for name, cats in groupmap.items():
        grouped = sum([h[{axis: name}] for name in cats])
        new[{axis: name}] = grouped.view(flow=True)
    return new

groupby(hn, {"d1": ["A", "B", "C"], 'd2': ["D", "E", 'F']})[{'tag': s[0.5j:len:sum], 'region': 0, 'syst': "X"}].plot();
plt.legend();

### Desired end goal - 1D templates of each sample, passing a cut, per region per systematic

In [ ]:
cut = {'tag': s[0.5j:len:sum]} # Events passing 0.5 threshold

templates = {}
for sample in hn.axes['dataset']:
    for region in hn.axes['region']:
        for syst in hn.axes['syst']:
            template_name = f"region{region}_{sample}_sys{syst}"
            templates[template_name] = hn[{**cut, 'dataset': sample, 'region': region, 'syst': syst}]

In [ ]:
templates['region0_B_sysX']

### Save it via uproot

In [ ]:
import uproot 
fout = uproot.recreate("some_file.root")
fout["my_hist"] = templates['region0_B_sysX']
fout.close()

In [ ]:
fin = uproot.open("some_file.root")
hep.histplot(fin['my_hist'])

# Styling with mplhep
- Primary purpose of `mplhep` is to serve and distribute styles 
    - **ALICE**
    - **ATLAS**
    - **CMS**
    - **LHCb**
- To ensure plots looks the same on any framework fonts need to be included
 - I am liable to go on a rant, so suffice to say:
 - We package an open look-alike of Helvetica called Tex Gyre Heros

In [ ]:
hep.style.use([hep.style.CMS, {'figure.figsize': (8, 8)}])
hep.histplot(np.histogram(np.random.normal(10, 3, 1000)), histtype='fill');
hep.cms.label();

# CMS Colors - automatically with `hep.style.CMS`

- Data should be always shown in black. Basic color recommendations with examples are found below.

- Categorical Data (e.g. 1D Stackplots): Use the color scheme suggested by M. Petroff in arXiv:2107.02270v2 and available on GitHub (MIT License). 
- Specifically you should use the 6-color scheme:
`["#5790fc", "#f89c20", "#e42536", "#964a8b", "#9c9ca1", "#7a21dd"]`

In [ ]:
from matplotlib.colors import ListedColormap
petroff6 = ListedColormap(["#5790fc", "#f89c20", "#e42536", "#964a8b", "#9c9ca1", "#7a21dd"])
petroff6

In [ ]:
hn[{'tag': s[0.5j:len:sum], 'region': 0, 'syst': "X", 'x': s[::hist.tag.rebin(3)]}].plot(histtype='fill', stack=True);
plt.legend();

 - or if more colors are needed the 10-color scheme:
    ```
    ["#3f90da", "#ffa90e", "#bd1f01", "#94a4a2", "#832db6", "#a96b59", "#e76300", "#b9ac70", "#717581", "#92dadd"]
    ```

In [ ]:
from matplotlib.colors import ListedColormap
petroff10 = ListedColormap(["#3f90da", "#ffa90e", "#bd1f01", "#94a4a2", "#832db6", "#a96b59", "#e76300", "#b9ac70", "#717581", "#92dadd"])
petroff10

# 2D plot

In [ ]:
hn[{'tag': s[0.5j:len:sum], 'region': 0, 'syst': "X", 'x': s[::hist.tag.rebin(3)]}].plot2d()